In [2]:
import os
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

from scipy.special import expit 

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
import plotly.io as pio
from plotly.subplots import make_subplots

pyo.init_notebook_mode(connected = True)


from sklearn.metrics import roc_auc_score, f1_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import (
    GridSearchCV,
    train_test_split,
    cross_val_score,
    GroupShuffleSplit,
    permutation_test_score,
    StratifiedKFold,
    KFold,
    cross_validate
)

from tqdm import tqdm 


from sklearn.ensemble import ExtraTreesClassifier, RandomForestRegressor

from merf import MERF
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from pymer4.models import Lmer


import sys
sys.path.insert(0, '../')
from utils import multivariate_classifier, correct_name_markers, plot_univariate

INFO     [situation.py:41] cffi mode is CFFI_MODE.ANY
INFO     [situation.py:218] R home found: /Library/Frameworks/R.framework/Resources
INFO     [situation.py:161] R library path: 
INFO     [situation.py:165] LD_LIBRARY_PATH: 
INFO     [embedded.py:20] Default options to initialize R: rpy2, --quiet, --no-save
INFO     [embedded.py:269] R is already initialized. No need to initialize.


Using MNE with API > 0.19


In [3]:
# plotting parameters
grey = "#21201F"
green = "#9AC529"
lblue = "#42B9B2"
pink = "#DE237B"
orange = "#F38A31"

nt_colors = [green, lblue, pink, orange]

plt.style.use("ggplot")
fig_width = 12  # width in inches
fig_height = 9  # height in inches
fig_size = [fig_width, fig_height]
plt.rcParams["figure.figsize"] = fig_size
plt.rcParams["figure.autolayout"] = True

sns.set(
    style="white",
    context="notebook",
    font_scale=1,
    rc={
        "axes.labelcolor": grey,
        "text.color": grey,
        "axes.edgecolor": grey,
        "xtick.color": grey,
        "ytick.color": grey,
    },
)

sns.set_palette(sns.color_palette
(nt_colors))

# Load Data
Loads data from the computed markers. From `Data` directory

In [4]:
data_path = "../../Data/"
results_path = "../../Results/"
fig_path = "../../Results/Figs/"

df = pd.read_csv(os.path.join(data_path, 'all_markers.csv'), index_col = 0)

In [5]:
markers = ['wSMI_1', 'wSMI_2', 'wSMI_4', 'wSMI_8', 'p_e_1', 'p_e_2',
       'p_e_4', 'p_e_8', 'k', 'se','msf', 'sef90', 'sef95', 'b', 'b_n', 'g',
       'g_n', 't', 't_n', 'd', 'd_n', 'a_n', 'a', 'CNV', 'P1', 'P3a', 'P3b',]
#           'ft', 'ft_n']
erps =['CNV', 'P1', 'P3a', 'P3b']
# erps = [r'$CNV$', r'$P1$', r'$P3a$',r'$P3b$']

# markers =  [r'$\delta$',r'$|\delta|$',r'$\theta$', r'$|\theta|$',r'$\alpha$', r'$|\alpha|$',r'$\beta$', r'$|\beta|$',r'$\gamma$', r'$|\gamma|$',
#             r'$PE\gamma$',r'$PE\beta$',r'$PE\alpha$',r'$PE\theta$',
#             r'$wSMI\gamma$',r'$wSMI\beta$',r'$wSMI\alpha$',r'$wSMI\theta$', 
#             r'$K$',r'$SE$',r'$MSF$', r'$SEF90$', r'$SEF95$', 
#             r'$CNV$', r'$P1$', r'$P3a$',r'$P3b$'
#            ]


# df_subtracted = df.query("preproc == 'subtracted'").drop(columns = erps+['preproc'])
# df_erp = df.query("preproc == 'erp'").drop(columns = np.setdiff1d(markers,erps).tolist()+['preproc'])

# df_markers = df_subtracted.merge(df_erp, 'inner', on =np.setdiff1d(df_subtracted.columns, markers).tolist() )

df_markers = (df
              .query("stimuli == 'go'")
              .query("correct == 'correct'")
              .query('prev_trial < 5')
              .drop(['stimuli', 'correct', 'prev_trial', 'label', 'events',  'epoch_type', 'preproc', 'ft', 'ft_n'], axis = 1)
              .query("mind in ['on-task','dMW', 'sMW']")
              .groupby(['segment', 'participant']).filter(lambda x: len(x) > 1)
             )

df_markers['segment'] = df_markers['segment'].str.replace('s', '').astype(int)

# By segment Mulivariate analysis

## On-task Vs Mind- Wandering
This can only be performed for PC probes  as they are the only ones with On-task reports.


Prepares the dataframe for the analysis

In [22]:
agg_dict = {k:['mean', 'std'] for k in markers }
agg_dict.update({k:'first' for k in df_markers.drop(markers, axis=1).columns})

df_mind = (
    df_markers
    .query("probe == 'PC'")
    .groupby(['segment', 'participant'], as_index = False).agg(agg_dict)
    .assign(
    mind2 = lambda df: np.where(df.mind == 'on-task', 'on-task', 'mw'))
)

############################################################
################ Use normal names################
############################################################

df_mind.columns = df_mind.columns.map("_".join)

df_mind  = (df_mind
            .rename(columns = {'participant_first':'participant', 'probe_first':'probe', 'mind_first':'mind', 'segment_first':'segment', 'mind2_':'mind2'})
#             .query("mind != 'dMW'") #if you want to test against just one of the mw            
            .drop([ 'probe', 'mind',], axis = 1) 
           )

############################################################
################ Use latex command for nmaes################
############################################################

##it slow downs the computer, just for final figures.

# df_mind = correct_name_markers(df_mind)

# df_mind.columns = df_mind.columns.map("$_{".join).map(lambda x: x + '}$').map(lambda x: x.replace('$$', ''))

# df_mind  = (df_mind
#             .rename(columns = {'participant$_{first}$':'participant', 'probe$_{first}$':'probe', 'mind$_{first}$':'mind', 'segment$_{first}$':'segment', 'mind2$_{}$':'mind2'})
# #             .query("mind != 'dMW'") #if you want to test against just one of the mw            
#             .drop(['probe', 'mind', 'segment'], axis = 1) 
        #    )
        
df_mind['mind2_numeric'] = (df_mind['mind2'] == 'mw').astype(int)


In [74]:
from pymer4.models import Lmer
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from scipy.special import expit  # For the logistic sigmoid function

# Prepare data
X = df_mind.drop(['mind2', 'mind2_numeric', 'participant'], axis=1)
clusters = df_mind['participant']
y = df_mind['mind2_numeric']

selected_features = vif_data[vif_data["VIF"] <= 10]["feature"].tolist()
if 'const' in selected_features:
    selected_features.remove('const')

X = X[selected_features]

# Generate fixed effects formula dynamically
fixed_effects_formula = ' + '.join(X.columns)

# Initialize StratifiedKFold, and lists for actual AUC and permutation AUC scores
skf = StratifiedKFold(n_splits=5)
actual_auc_scores = []
perm_auc_scores_all = []

for train_index, test_index in tqdm(skf.split(X, y), desc="KFold Iteration"):
    df_train = df_mind.iloc[train_index]
    df_test = df_mind.iloc[test_index]
    y_test = y.iloc[test_index]
    
    # Fitting the GLMM model
    formula = f"mind2_numeric ~ {fixed_effects_formula} + (1|participant)"
    model = Lmer(formula, data=df_train, family="binomial")
    model.fit(verbose=False)
    
    # Make predictions and compute AUC
    predicted_probabilities = model.predict(df_test, use_rfx=True, verify_predictions=False)
    auc = roc_auc_score(y_test, predicted_probabilities)
    actual_auc_scores.append(auc)
    
    # Initialize list to store permutation AUC scores for this fold
    perm_auc_scores = []
    # Permutation test (500 permutations)
    for i in tqdm(range(100), desc="Permutation Test"):
        y_perm = np.random.permutation(df_train['mind2_numeric'])
        model_perm = Lmer(formula, data=df_train.assign(mind2_numeric=y_perm), family="binomial")
        model_perm.fit(verbose=False)
        
        # Make predictions and compute AUC for permuted labels
        predicted_probabilities_perm = model_perm.predict(df_test, use_rfx=True, verify_predictions=False)
        perm_auc = roc_auc_score(y_test, predicted_probabilities_perm)
        perm_auc_scores.append(perm_auc)
    
    # Add the list of scores for this fold to the master list
    perm_auc_scores_all.append(perm_auc_scores)

# Convert 2D list to DataFrame
df_perm_mind = pd.DataFrame(perm_auc_scores_all).T

# Average across folds for each permutation run
df_perm_mind['Mean_AUC'] = df_perm_mind.mean(axis=1)

# Save DataFrames to CSV
# Assuming results_path is defined
df_auc_mind = pd.DataFrame({'auc': actual_auc_scores, 'Model': 'Actual'})
df_auc_mind.to_csv(os.path.join(results_path, 'GLMM_mvpa_actual_mind.csv'), index=False)
df_perm_mind.to_csv(os.path.join(results_path, 'GLMM_mvpa_perm_mind.csv'), index=False)

KFold Iteration: 0it [00:00, ?it/s]

[1] "failure to converge in 10000 evaluations"
[2] " \n"                                     

unable to evaluate scaled gradient 

 Hessian is numerically singular: parameters are not uniquely determined 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: mind2_numeric~wSMI_1_mean+wSMI_1_std+wSMI_2_mean+wSMI_2_std+wSMI_4_mean+wSMI_4_std+wSMI_8_mean+wSMI_8_std+p_e_1_std+p_e_2_std+p_e_4_std+p_e_8_std+k_std+se_std+msf_std+b_std+b_n_std+g_std+g_n_std+t_std+t_n_std+d_std+d_n_std+a_n_std+a_std+CNV_mean+CNV_std+P1_mean+P1_std+P3a_mean+P3a_std+P3b_mean+P3b_std+(1|participant)

Family: binomial	 Inference: parametric

Number of observations: 140	 Groups: {'participant': 24.0}

Log-likelihood: -70.293 	 AIC: 210.587

Random effects:

                    Name    Var    Std
participant  (Intercept)  1.582  1.258

No random effect correlations specified

Fixed effects:



boundary (singular) fit: see ?isSingular 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: mind2_numeric~wSMI_1_mean+wSMI_1_std+wSMI_2_mean+wSMI_2_std+wSMI_4_mean+wSMI_4_std+wSMI_8_mean+wSMI_8_std+p_e_1_std+p_e_2_std+p_e_4_std+p_e_8_std+k_std+se_std+msf_std+b_std+b_n_std+g_std+g_n_std+t_std+t_n_std+d_std+d_n_std+a_n_std+a_std+CNV_mean+CNV_std+P1_mean+P1_std+P3a_mean+P3a_std+P3b_mean+P3b_std+(1|participant)

Family: binomial	 Inference: parametric

Number of observations: 140	 Groups: {'participant': 24.0}

Log-likelihood: -77.922 	 AIC: 225.845

Random effects:

                    Name  Var  Std
participant  (Intercept)  0.0  0.0

No random effect correlations specified

Fixed effects:

[1] "failure to converge in 10000 evaluations"
[2] " \n"                                     

unable to evaluate scaled gradient 

Model failed to converge: degenerate  Hessian with 4 negative eigenvalues 



Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: mind2_numeric~wSMI_1_mean+wSMI_1_std+wSMI_2_mean+wSMI_2_std+wSMI_4_mean+wSMI_4_std+wSMI_8_mean+wSMI_8_std+p_e_1_std+p_e_2_std+p_e_4_std+p_e_8_std+k_std+se_std+msf_std+b_std+b_n_std+g_std+g_n_std+t_std+t_n_std+d_std+d_n_std+a_n_std+a_std+CNV_mean+CNV_std+P1_mean+P1_std+P3a_mean+P3a_std+P3b_mean+P3b_std+(1|participant)

Family: binomial	 Inference: parametric

Number of observations: 140	 Groups: {'participant': 24.0}

Log-likelihood: -70.637 	 AIC: 211.274

Random effects:

                    Name    Var    Std
participant  (Intercept)  0.746  0.863

No random effect correlations specified

Fixed effects:

boundary (singular) fit: see ?isSingular 



Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: mind2_numeric~wSMI_1_mean+wSMI_1_std+wSMI_2_mean+wSMI_2_std+wSMI_4_mean+wSMI_4_std+wSMI_8_mean+wSMI_8_std+p_e_1_std+p_e_2_std+p_e_4_std+p_e_8_std+k_std+se_std+msf_std+b_std+b_n_std+g_std+g_n_std+t_std+t_n_std+d_std+d_n_std+a_n_std+a_std+CNV_mean+CNV_std+P1_mean+P1_std+P3a_mean+P3a_std+P3b_mean+P3b_std+(1|participant)

Family: binomial	 Inference: parametric

Number of observations: 140	 Groups: {'participant': 24.0}

Log-likelihood: -83.594 	 AIC: 237.187

Random effects:

                    Name  Var  Std
participant  (Intercept)  0.0  0.0

No random effect correlations specified

Fixed effects:



...

boundary (singular) fit: see ?isSingular 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: mind2_numeric~wSMI_1_mean+wSMI_1_std+wSMI_2_mean+wSMI_2_std+wSMI_4_mean+wSMI_4_std+wSMI_8_mean+wSMI_8_std+p_e_1_std+p_e_2_std+p_e_4_std+p_e_8_std+k_std+se_std+msf_std+b_std+b_n_std+g_std+g_n_std+t_std+t_n_std+d_std+d_n_std+a_n_std+a_std+CNV_mean+CNV_std+P1_mean+P1_std+P3a_mean+P3a_std+P3b_mean+P3b_std+(1|participant)

Family: binomial	 Inference: parametric

Number of observations: 141	 Groups: {'participant': 21.0}

Log-likelihood: -78.203 	 AIC: 226.406

Random effects:

                    Name  Var  Std
participant  (Intercept)  0.0  0.0

No random effect correlations specified

Fixed effects:



boundary (singular) fit: see ?isSingular 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: mind2_numeric~wSMI_1_mean+wSMI_1_std+wSMI_2_mean+wSMI_2_std+wSMI_4_mean+wSMI_4_std+wSMI_8_mean+wSMI_8_std+p_e_1_std+p_e_2_std+p_e_4_std+p_e_8_std+k_std+se_std+msf_std+b_std+b_n_std+g_std+g_n_std+t_std+t_n_std+d_std+d_n_std+a_n_std+a_std+CNV_mean+CNV_std+P1_mean+P1_std+P3a_mean+P3a_std+P3b_mean+P3b_std+(1|participant)

Family: binomial	 Inference: parametric

Number of observations: 141	 Groups: {'participant': 21.0}

Log-likelihood: -82.804 	 AIC: 235.608

Random effects:

                    Name  Var  Std
participant  (Intercept)  0.0  0.0

No random effect correlations specified

Fixed effects:



[1] "failure to converge in 10000 evaluations"
[2] " \n"                                     

unable to evaluate scaled gradient 

Model failed to converge: degenerate  Hessian with 5 negative eigenvalues 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: mind2_numeric~wSMI_1_mean+wSMI_1_std+wSMI_2_mean+wSMI_2_std+wSMI_4_mean+wSMI_4_std+wSMI_8_mean+wSMI_8_std+p_e_1_std+p_e_2_std+p_e_4_std+p_e_8_std+k_std+se_std+msf_std+b_std+b_n_std+g_std+g_n_std+t_std+t_n_std+d_std+d_n_std+a_n_std+a_std+CNV_mean+CNV_std+P1_mean+P1_std+P3a_mean+P3a_std+P3b_mean+P3b_std+(1|participant)

Family: binomial	 Inference: parametric

Number of observations: 141	 Groups: {'participant': 21.0}

Log-likelihood: -72.927 	 AIC: 215.854

Random effects:

                    Name    Var    Std
participant  (Intercept)  0.288  0.537

No random effect correlations specified

Fixed effects:

boundary (singular) fit: see ?isSingular 



Permutation Test: 100%|██████████| 100/100 [08:07<00:00,  4.87s/it]
KFold Iteration: 5it [53:47, 645.45s/it]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: mind2_numeric~wSMI_1_mean+wSMI_1_std+wSMI_2_mean+wSMI_2_std+wSMI_4_mean+wSMI_4_std+wSMI_8_mean+wSMI_8_std+p_e_1_std+p_e_2_std+p_e_4_std+p_e_8_std+k_std+se_std+msf_std+b_std+b_n_std+g_std+g_n_std+t_std+t_n_std+d_std+d_n_std+a_n_std+a_std+CNV_mean+CNV_std+P1_mean+P1_std+P3a_mean+P3a_std+P3b_mean+P3b_std+(1|participant)

Family: binomial	 Inference: parametric

Number of observations: 141	 Groups: {'participant': 21.0}

Log-likelihood: -79.977 	 AIC: 229.954

Random effects:

                    Name  Var  Std
participant  (Intercept)  0.0  0.0

No random effect correlations specified

Fixed effects:



In [16]:
df_auc_mind = pd.read_csv(os.path.join(results_path, 'GLMM_mvpa_actual_mind.csv'))
df_perm_mind = pd.read_csv(os.path.join(results_path, 'GLMM_mvpa_perm_mind.csv'))

print(df_auc_mind.auc.mean())

fig =px.violin(df_auc_mind, x = 'auc', box = True, points = 'all',template = "plotly_white", color_discrete_sequence = [pink], labels = {'auc': 'AUC'})
fig.add_vline(x=0.5, line_width=3, line_dash="dash", line_color="grey")

fig.update_layout(
#     title = f'Mean AUC = {df_forest_mind.auc.mean()}',
    autosize=False,
    width=800,
    height=800,
    yaxis = {'title': 'Whole model',
            'showticklabels': True,
            'tickmode': 'linear',},
    xaxis ={
             'range':[0.2, 1]
        }
)
fig.show()

fig.write_image(os.path.join(fig_path, 'multivariate_glmm_cv_mind.png'))

0.5947583419332645


In [15]:
# Calculate p-value
pvalue_mind = np.mean(df_perm_mind['Mean_AUC'] >= np.mean(df_auc_mind['auc']))

df = pd.concat([df_perm_mind, df_auc_mind])
fig = px.histogram(df_perm_mind, x='Mean_AUC',nbins = 25,  color_discrete_sequence = [pink])

fig.add_vline(x=df_auc_mind.auc.mean(), line_width=3, line_dash="dash", line_color="grey")

fig.add_annotation(x=df_auc_mind.auc.mean() + 0.1, y=5,
            text=f"Score on original labels: <br>Mean AUC: {df_auc_mind.auc.mean():.3f}<br>p = {pvalue_mind:.3f}",
            showarrow=False,
            arrowhead=1,
            align = 'left',
            )


fig.update_layout(
#     title = f'Mean AUC = {df_forest_mind.auc.mean()}',
    autosize=False,
    template = "plotly_white",
    font=dict(
        family="Times new roman",
        size=20,
        color="black"
    ),
    width=800,
    height=800,
    bargap=0.05,
    yaxis = {'title': '',
             'tickfont': {"size": 20},
    },
    xaxis ={'title':'CV AUC',
             'range':[0.25, 0.75],
             'tickfont':{"size": 20}
        }
)
fig.show()

fig.write_image(os.path.join(fig_path, 'multivariate_glmm_perm_mind.png'))
fig.write_image(os.path.join(fig_path, 'multivariate_glmm_perm_mind.svg'))

In [ ]:
top_10 = feat_import_mind.sort_values(by = 'mean_importance').tail(10)
fig = px.scatter(top_10,x = 'mean_importance', y = 'features', template = "plotly_white",
                  color_discrete_sequence = [pink],

                 category_orders = {'significant': ['p > 0.05','p < 0.05 uncorrected', 'p < 0.05 FDR corrected']}, 
                 labels = {'value':'Feature importance', 'features': 'Markers'}

                )

fig.update_traces(marker=dict(size = 8))

fig.update_layout(
    width=650,
    height=900,
#     autosize = True, 
    template = 'plotly_white',
        font=dict(
        family="Times new roman",
        size=20,
        color="black"
    ),
    xaxis = dict(
            visible=True,
            # range = [0.37,0.63], 
            tickfont = {"size": 20},
            title = 'Feature Importance'
        ),
    yaxis = dict(
        tickfont = {"size": 20},
        # autorange = False,    
        automargin = True,
        # range = [-1,len(feat_import_mw)],
        dtick = 1
        ),
    showlegend=True, 

)

fig.show()
# pio.write_json(fig, 'Figs/univariate_roc_mw_segment.plotly')
fig.write_image(os.path.join(fig_path, 'feat_importance_mind.png'))
fig.write_image(os.path.join(fig_path, 'feat_importance_mind.svg'))

## dMW Vs sMW
This will be only performed in SC as they have more trials

In [46]:
agg_dict = {k:['mean', 'std'] for k in markers }
agg_dict.update({k:'first' for k in df_markers.drop(markers, axis=1).columns})

df_mw = (
    df_markers
    .query("probe == 'SC'")
    .query("mind != 'on-task'")
    .groupby(['segment', 'participant'], as_index = False).agg(agg_dict)
)

############################################################
################ Use normal names################
############################################################
df_mw.columns = df_mw.columns.map("_".join)

df_mw  = (df_mw
            .rename(columns = {'participant_first':'participant', 'probe_first':'probe', 'mind_first':'mind', 'segment_first':'segment'})
            .drop([ 'probe',], axis = 1) 
           )


############################################################
################ Use latex command for nmaes################
############################################################

# df_mw = correct_name_markers(df_mw)

# df_mw.columns = df_mw.columns.map("$_{".join).map(lambda x: x + '}$').map(lambda x: x.replace('$$', ''))

# df_mw  = (df_mw
#             .rename(columns = {'participant$_{first}$':'participant', 'probe$_{first}$':'probe', 'mind$_{first}$':'mind', 'segment$_{first}$':'segment', 'mind$_{}$':'mind'})
# #             .query("mind != 'dMW'") #if you want to test against just one of the mw   
#             .drop(['participant', 'probe',  'segment'], axis = 1)

#            )


df_mw['mind_numeric'] = (df_mw['mind'] == 'sMW').astype(int)

df_mw.to_csv(os.path.join(results_path,'data_mw.csv'))


In [83]:
from pymer4.models import Lmer
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
from scipy.special import expit  # For the logistic sigmoid function

# Prepare data
X = df_mw.drop(['mind', 'mind_numeric', 'participant'], axis=1)
clusters = df_mw['participant']
y = df_mw['mind_numeric']

selected_features = vif_data[vif_data["VIF"] <= 100]["feature"].tolist()
if 'const' in selected_features:
    selected_features.remove('const')

X = X[selected_features]

# Generate fixed effects formula dynamically
fixed_effects_formula = ' + '.join(X.columns)

# Initialize StratifiedKFold, and lists for actual AUC and permutation AUC scores
skf = StratifiedKFold(n_splits=5)
actual_auc_scores = []
perm_auc_scores_all = []
optimal_cutoffs = []

for train_index, test_index in tqdm(skf.split(X, y), desc="KFold Iteration"):
    df_train = df_mw.iloc[train_index]
    df_test = df_mw.iloc[test_index]
    y_test = y.iloc[test_index]
    
    # Fitting the GLMM model
    formula = f"mind_numeric ~ {fixed_effects_formula} + (1|participant)"
    model = Lmer(formula, data=df_train, family="binomial")
    model.fit(verbose=False)
    
    # Make predictions and compute AUC
    predicted_probabilities = model.predict(df_test, use_rfx=True, verify_predictions=False)
    auc = roc_auc_score(y_test, predicted_probabilities)
    actual_auc_scores.append(auc)
    
    # Initialize list to store permutation AUC scores for this fold
    perm_auc_scores = []
    
    # Initialize list to store permutation AUC scores for this fold
    perm_auc_scores = []
    # Permutation test (500 permutations)
    for i in tqdm(range(100), desc="Permutation Test"):
        y_perm = np.random.permutation(df_train['mind_numeric'])
        model_perm = Lmer(formula, data=df_train.assign(mind_numeric=y_perm), family="binomial")
        model_perm.fit(verbose=False)
        
        # Make predictions and compute AUC for permuted labels
        predicted_probabilities_perm = model_perm.predict(df_test, use_rfx=True, verify_predictions=False)
        perm_auc = roc_auc_score(y_test, predicted_probabilities_perm)
        perm_auc_scores.append(perm_auc)
    
    # Add the list of scores for this fold to the master list
    perm_auc_scores_all.append(perm_auc_scores)

# Convert 2D list to DataFrame
df_perm_mw = pd.DataFrame(perm_auc_scores_all).T

# Average across folds for each permutation run
df_perm_mw['Mean_AUC'] = df_perm_mw.mean(axis=1)

# Save DataFrames to CSV
# Assuming results_path is defined
df_auc_mw = pd.DataFrame({'auc': actual_auc_scores, 'Model': 'Actual'})
df_auc_mw.to_csv(os.path.join(results_path, 'GLMM_mvpa_actual_mw.csv'), index=False)
df_perm_mw.to_csv(os.path.join(results_path, 'GLMM_mvpa_perm_mw.csv'), index=False)

KFold Iteration: 0it [00:00, ?it/s]

[1] "failure to converge in 10000 evaluations"
[2] " \n"                                     

unable to evaluate scaled gradient 

Model failed to converge: degenerate  Hessian with 3 negative eigenvalues 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: mind_numeric~wSMI_1_mean+wSMI_1_std+wSMI_2_mean+wSMI_2_std+wSMI_4_mean+wSMI_4_std+wSMI_8_mean+wSMI_8_std+p_e_1_mean+p_e_1_std+p_e_2_mean+p_e_2_std+p_e_4_mean+p_e_4_std+p_e_8_mean+p_e_8_std+k_mean+k_std+se_mean+se_std+msf_std+sef90_std+sef95_std+b_std+b_n_std+g_std+g_n_std+t_std+t_n_std+d_mean+d_std+d_n_std+a_n_std+a_std+CNV_mean+CNV_std+P1_mean+P1_std+P3a_mean+P3a_std+P3b_mean+P3b_std+(1|participant)

Family: binomial	 Inference: parametric

Number of observations: 428	 Groups: {'participant': 19.0}

Log-likelihood: -237.035 	 AIC: 562.071

Random effects:

                    Name    Var   Std
participant  (Intercept)  1.299  1.14

No random effect correlations specified

Fixed effects:



boundary (singular) fit: see ?isSingular 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: mind_numeric~wSMI_1_mean+wSMI_1_std+wSMI_2_mean+wSMI_2_std+wSMI_4_mean+wSMI_4_std+wSMI_8_mean+wSMI_8_std+p_e_1_mean+p_e_1_std+p_e_2_mean+p_e_2_std+p_e_4_mean+p_e_4_std+p_e_8_mean+p_e_8_std+k_mean+k_std+se_mean+se_std+msf_std+sef90_std+sef95_std+b_std+b_n_std+g_std+g_n_std+t_std+t_n_std+d_mean+d_std+d_n_std+a_n_std+a_std+CNV_mean+CNV_std+P1_mean+P1_std+P3a_mean+P3a_std+P3b_mean+P3b_std+(1|participant)

Family: binomial	 Inference: parametric

Number of observations: 428	 Groups: {'participant': 19.0}

Log-likelihood: -279.199 	 AIC: 646.398

Random effects:

                    Name  Var  Std
participant  (Intercept)  0.0  0.0

No random effect correlations specified

Fixed effects:



boundary (singular) fit: see ?isSingular 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: mind_numeric~wSMI_1_mean+wSMI_1_std+wSMI_2_mean+wSMI_2_std+wSMI_4_mean+wSMI_4_std+wSMI_8_mean+wSMI_8_std+p_e_1_mean+p_e_1_std+p_e_2_mean+p_e_2_std+p_e_4_mean+p_e_4_std+p_e_8_mean+p_e_8_std+k_mean+k_std+se_mean+se_std+msf_std+sef90_std+sef95_std+b_std+b_n_std+g_std+g_n_std+t_std+t_n_std+d_mean+d_std+d_n_std+a_n_std+a_std+CNV_mean+CNV_std+P1_mean+P1_std+P3a_mean+P3a_std+P3b_mean+P3b_std+(1|participant)

Family: binomial	 Inference: parametric

Number of observations: 428	 Groups: {'participant': 19.0}

Log-likelihood: -277.257 	 AIC: 642.513

Random effects:

                    Name  Var  Std
participant  (Intercept)  0.0  0.0

No random effect correlations specified

Fixed effects:



boundary (singular) fit: see ?isSingular 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: mind_numeric~wSMI_1_mean+wSMI_1_std+wSMI_2_mean+wSMI_2_std+wSMI_4_mean+wSMI_4_std+wSMI_8_mean+wSMI_8_std+p_e_1_mean+p_e_1_std+p_e_2_mean+p_e_2_std+p_e_4_mean+p_e_4_std+p_e_8_mean+p_e_8_std+k_mean+k_std+se_mean+se_std+msf_std+sef90_std+sef95_std+b_std+b_n_std+g_std+g_n_std+t_std+t_n_std+d_mean+d_std+d_n_std+a_n_std+a_std+CNV_mean+CNV_std+P1_mean+P1_std+P3a_mean+P3a_std+P3b_mean+P3b_std+(1|participant)

Family: binomial	 Inference: parametric

Number of observations: 428	 Groups: {'participant': 19.0}

Log-likelihood: -275.895 	 AIC: 639.790

Random effects:

                    Name  Var  Std
participant  (Intercept)  0.0  0.0

No random effect correlations specified

Fixed effects:



...

boundary (singular) fit: see ?isSingular 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: mind_numeric~wSMI_1_mean+wSMI_1_std+wSMI_2_mean+wSMI_2_std+wSMI_4_mean+wSMI_4_std+wSMI_8_mean+wSMI_8_std+p_e_1_mean+p_e_1_std+p_e_2_mean+p_e_2_std+p_e_4_mean+p_e_4_std+p_e_8_mean+p_e_8_std+k_mean+k_std+se_mean+se_std+msf_std+sef90_std+sef95_std+b_std+b_n_std+g_std+g_n_std+t_std+t_n_std+d_mean+d_std+d_n_std+a_n_std+a_std+CNV_mean+CNV_std+P1_mean+P1_std+P3a_mean+P3a_std+P3b_mean+P3b_std+(1|participant)

Family: binomial	 Inference: parametric

Number of observations: 429	 Groups: {'participant': 22.0}

Log-likelihood: -271.588 	 AIC: 631.176

Random effects:

                    Name  Var  Std
participant  (Intercept)  0.0  0.0

No random effect correlations specified

Fixed effects:



boundary (singular) fit: see ?isSingular 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: mind_numeric~wSMI_1_mean+wSMI_1_std+wSMI_2_mean+wSMI_2_std+wSMI_4_mean+wSMI_4_std+wSMI_8_mean+wSMI_8_std+p_e_1_mean+p_e_1_std+p_e_2_mean+p_e_2_std+p_e_4_mean+p_e_4_std+p_e_8_mean+p_e_8_std+k_mean+k_std+se_mean+se_std+msf_std+sef90_std+sef95_std+b_std+b_n_std+g_std+g_n_std+t_std+t_n_std+d_mean+d_std+d_n_std+a_n_std+a_std+CNV_mean+CNV_std+P1_mean+P1_std+P3a_mean+P3a_std+P3b_mean+P3b_std+(1|participant)

Family: binomial	 Inference: parametric

Number of observations: 429	 Groups: {'participant': 22.0}

Log-likelihood: -272.522 	 AIC: 633.044

Random effects:

                    Name  Var  Std
participant  (Intercept)  0.0  0.0

No random effect correlations specified

Fixed effects:



boundary (singular) fit: see ?isSingular 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: mind_numeric~wSMI_1_mean+wSMI_1_std+wSMI_2_mean+wSMI_2_std+wSMI_4_mean+wSMI_4_std+wSMI_8_mean+wSMI_8_std+p_e_1_mean+p_e_1_std+p_e_2_mean+p_e_2_std+p_e_4_mean+p_e_4_std+p_e_8_mean+p_e_8_std+k_mean+k_std+se_mean+se_std+msf_std+sef90_std+sef95_std+b_std+b_n_std+g_std+g_n_std+t_std+t_n_std+d_mean+d_std+d_n_std+a_n_std+a_std+CNV_mean+CNV_std+P1_mean+P1_std+P3a_mean+P3a_std+P3b_mean+P3b_std+(1|participant)

Family: binomial	 Inference: parametric

Number of observations: 429	 Groups: {'participant': 22.0}

Log-likelihood: -284.761 	 AIC: 657.523

Random effects:

                    Name  Var  Std
participant  (Intercept)  0.0  0.0

No random effect correlations specified

Fixed effects:

boundary (singular) fit: see ?isSingular 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: mind_numeric~wSMI_1_mean+wSMI_1_std+wSMI_2_mean+wSMI_2_std+wSMI_4_mean+wSMI_4_std+wSMI_8

Permutation Test: 100%|██████████| 100/100 [18:36<00:00, 11.16s/it]
KFold Iteration: 5it [1:48:27, 1301.46s/it]

boundary (singular) fit: see ?isSingular 

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: mind_numeric~wSMI_1_mean+wSMI_1_std+wSMI_2_mean+wSMI_2_std+wSMI_4_mean+wSMI_4_std+wSMI_8_mean+wSMI_8_std+p_e_1_mean+p_e_1_std+p_e_2_mean+p_e_2_std+p_e_4_mean+p_e_4_std+p_e_8_mean+p_e_8_std+k_mean+k_std+se_mean+se_std+msf_std+sef90_std+sef95_std+b_std+b_n_std+g_std+g_n_std+t_std+t_n_std+d_mean+d_std+d_n_std+a_n_std+a_std+CNV_mean+CNV_std+P1_mean+P1_std+P3a_mean+P3a_std+P3b_mean+P3b_std+(1|participant)

Family: binomial	 Inference: parametric

Number of observations: 429	 Groups: {'participant': 22.0}

Log-likelihood: -272.250 	 AIC: 632.501

Random effects:

                    Name  Var  Std
participant  (Intercept)  0.0  0.0

No random effect correlations specified

Fixed effects:



In [18]:
df_forest_mw = pd.read_csv(os.path.join(results_path, 'GLMM_mvpa_actual_mw.csv'))
df_perm_mw = pd.read_csv(os.path.join(results_path, 'GLMM_mvpa_perm_mw.csv'))


print(df_forest_mw.auc.mean())

fig =px.violin(df_forest_mw, x = 'auc', box = True, points = 'all',template = "plotly_white", color_discrete_sequence = [lblue], labels = {'auc': 'AUC'})
fig.add_vline(x=0.5, line_width=3, line_dash="dash", line_color="grey")

fig.update_layout(
#     title = f'Mean AUC = {df_forest_mind.auc.mean()}',
    autosize=False,
    width=800,
    height=800,
    yaxis = {'title': 'Whole model',
            'showticklabels': True,
            'tickmode': 'linear',},
    xaxis ={
             'range':[0.2, 1]
        }
)
fig.show()

fig.write_image(os.path.join(fig_path, 'multivariate_glmm_cv_mw.png'))

0.6767755595886269


In [17]:
# Calculate p-value
pvalue_mw = np.mean(df_perm_mw['Mean_AUC'] >= np.mean(df_auc_mw['auc']))

df = pd.concat([df_perm_mw, df_auc_mw])
fig = px.histogram(df_perm_mw, x='Mean_AUC',nbins = 25,  color_discrete_sequence = [lblue])

fig.add_vline(x=df_auc_mw.auc.mean(), line_width=3, line_dash="dash", line_color="grey")

fig.add_annotation(x=df_auc_mw.auc.mean() + 0.1, y=5,
            text=f"Score on original labels: <br>Mean AUC: {df_auc_mw.auc.mean():.3f}<br>p = {pvalue_mw:.3f}",
            showarrow=False,
            arrowhead=1,
            align = 'left',
            )


fig.update_layout(
#     title = f'Mean AUC = {df_forest_mind.auc.mean()}',
    autosize=False,
    template = "plotly_white",
    font=dict(
        family="Times new roman",
        size=20,
        color="black"
    ),
    width=800,
    height=800,
    bargap=0.05,
    yaxis = {'title': '',
             'tickfont': {"size": 20},
    },
    xaxis ={'title':'CV AUC',
             'range':[0.25, 0.8],
             'tickfont':{"size": 20}
        }
)
fig.show()

fig.write_image(os.path.join(fig_path, 'multivariate_glmm_perm_mw.png'))
fig.write_image(os.path.join(fig_path, 'multivariate_glmm_perm_mw.svg'))